In [5]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


2024-02-25 14:53:58.421519: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 14:53:58.564634: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 14:53:58.564667: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 14:53:58.568782: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 14:53:58.568820: I external/local_xla/xla/stream_executor

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

df = pd.read_csv('diabetes_prediction_dataset.csv')
df = df[df['age']>=18]
df = df[df['bmi']<=40]
#Preprocess the data
numeric_col=[]
non_numeric_col=[]
for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        if(df[column].nunique()<5):
            non_numeric_col.append(column)
        else:
            numeric_col.append(column)
    else:
        non_numeric_col.append(column)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['smoking_history'] = df['smoking_history'].replace({'not current':'former','ever':'never'})
df_copy = df.copy()
for col in non_numeric_col:
    df[col]=le.fit_transform(df[col])



In [7]:
y = df['diabetes']
X = df.drop('diabetes', axis = 1)

In [8]:
scale_pos_weight = y.value_counts()[0] /  y.value_counts()[1]

In [74]:
#Normalize the data
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, stratify = y_test)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
combined_X_train = np.concatenate((X_train_scaled,X_val_scaled), axis = 0)
combined_y_train = np.concatenate((y_train,y_val), axis = 0)

In [27]:
X_train_scaled.shape

(99468, 8)

In [11]:
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Dropout, Input
from keras.saving import register_keras_serializable

@register_keras_serializable()
class DiabetesClassifier(Model):
    def __init__(self, data_input, num_of_dense_layers = 3, dense_number = 8, dropout = 0, l2 = 0.001, opt_threshold = 0):
        super(DiabetesClassifier, self).__init__()
        self.opt_threshold = opt_threshold
        self.num_of_dense_layers = num_of_dense_layers
        self.dense_number = dense_number
        self.dropout = dropout
        self.data_input = data_input
        self.l2 = l2
        self.model = self.build_model()
    
    def call(self, inputs):
        return self.model(inputs)
    
    def build_model(self):
        inp = Input(shape = self.data_input)
        reg = keras.regularizers.l2(self.l2)
        x = Dense(self.data_input, activation = 'relu', kernel_regularizer=reg)(inp)
        for i in range(self.num_of_dense_layers):
            x = Dense(units = self.dense_number, activation = 'relu', kernel_regularizer=reg)(x)
            x = BatchNormalization()(x)
            x = Dropout(self.dropout)(x)
        output = Dense(units =1, activation = 'sigmoid')(x)
        model = Model(inputs = inp, outputs = output)
        return model

In [12]:
from keras import metrics
import keras

In [ ]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, f1_score, precision_score, recall_score


def objective(trial):
    num_of_layers = trial.suggest_int('num_of_layers', 2, 3, step = 1)
    dense_num_of_neurons = trial.suggest_int('num_of_neurons', 5, 8, step = 1)
    dense_num_of_neurons = 2 ** dense_num_of_neurons
    dropout = trial.suggest_float('dropout', 0.2, 0.4)
    l2 = trial.suggest_float('l2', 0.001, 0.01, log = True)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01, log = True)
    scale_pos_weight_multiplier = trial.suggest_int('scale_pos_weight_multiplier', 1, 3)
    
    model = DiabetesClassifier(X_train_scaled.shape[1], num_of_dense_layers =num_of_layers, dense_number =dense_num_of_neurons, dropout = dropout, l2 = l2)
    tensorboard = keras.callbacks.TensorBoard(log_dir='logs')
    optimizer = keras.optimizers.Adam(learning_rate = learning_rate)
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = [metrics.AUC(), metrics.Precision(), metrics.Recall(), metrics.Accuracy()])
    epochs = trial.suggest_int('epochs', 20,100, step = 10)
    model.fit(X_train_scaled, y_train, validation_data = (X_val_scaled, y_val), batch_size =4092, epochs = 20, class_weight = {1: scale_pos_weight/scale_pos_weight_multiplier, 0: 1}, callbacks = [tensorboard])

    n_splits = 5  # Number of folds for StratifiedKFold
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    opt_thresholds = []
    f1_scores = []
    recall_lis = []
    precision_lis = []
    # Choose the beta value you want.
    beta = 1.45    
    for train_index, val_index in cv.split(combined_X_train, combined_y_train):
        # Split data into training and validation for the current fold
        X_train_curr, X_val_curr = combined_X_train[train_index], combined_X_train[val_index]
        y_train_curr, y_val_curr = combined_y_train[train_index], combined_y_train[val_index]
        # Predict probabilities for the positive class on the validation data
        y_pred_prob = model.predict(X_val_curr).ravel()
    
        # Compute precision-recall curve
        precision, recall, thresholds = precision_recall_curve(y_val_curr, y_pred_prob)
    
        # Calculate F1 scores for each threshold
        # f1_scores_fold = [2 * (p * r) / (p + r) if (p + r) > 0 else 0 for p, r in zip(precision, recall)]
        f1_scores_fold = [(1 + beta**2) * (prec * rec) / ((beta**2 * prec) + rec) for prec, rec in zip(precision, recall)]

    # Find the index of the maximum F1 score
        opt_idx = np.argmax(f1_scores_fold)
        opt_threshold = thresholds[opt_idx] if opt_idx < len(thresholds) else 1.0
        
        # Store the optimal threshold for this fold
        opt_thresholds.append(opt_threshold)
        f1_scores.append(f1_scores_fold[opt_idx])
        y_pred = (y_pred_prob > opt_threshold).astype(int)
        recall_lis.append(recall_score(y_val_curr, y_pred))
        precision_lis.append(precision_score(y_val_curr, y_pred))
    
    average_opt_threshold = np.mean(opt_thresholds)
    average_f1_score = np.mean(f1_scores)
    trial.set_user_attr('average_opt_threshold', average_opt_threshold)
    trial.set_user_attr('Recall', np.mean(recall_lis))
    trial.set_user_attr('Precision', np.mean(precision_lis))
    return average_f1_score
    
study = optuna.create_study(direction='maximize')
study.sampler = optuna.samplers.TPESampler(multivariate=True)
study.optimize(objective, n_trials=40)

print('Number of finished trials:', len(study.trials))
print('Best result: ', study.best_trial.value)
print('Best trial:', study.best_trial.params)
# To use the best parameters:
best_params = study.best_trial.params
best_threshold = study.best_trial.user_attrs['average_opt_threshold']
df = study.trials_dataframe()
# Save to a CSV file for further analysis
df.to_csv("optuna_trials.csv", index=False, mode = 'a')


In [60]:
# extracted the best params from the CSV file
dropout, l2, learning_rate, num_of_layers, num_of_neurons, scale_pos_weight_multiplier = 0.283041005,	0.002680608,	0.001094543,	3,	7,	2
best_params = {'dropout':dropout, 'l2': l2, 'learning_rate':learning_rate,'num_of_layers': num_of_layers, 'num_of_neurons':num_of_neurons, 'scale_pos_weight_multiplier':scale_pos_weight_multiplier}


In [75]:
model = DiabetesClassifier(X_train_scaled.shape[1], num_of_dense_layers =best_params['num_of_layers'], dense_number = 2 ** best_params['num_of_neurons'], dropout = best_params['dropout'], l2 = best_params['l2'])
tensorboard = keras.callbacks.TensorBoard(log_dir='logs')
optimizer = keras.optimizers.Adam(learning_rate = best_params['learning_rate'])
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = [metrics.AUC(), metrics.Precision(), metrics.Recall(), metrics.Accuracy()])
model.fit(X_train_scaled, y_train, validation_data = (X_val_scaled, y_val), batch_size =4092, epochs = 80, class_weight = {1: scale_pos_weight/best_params['scale_pos_weight_multiplier'], 0: 1}, callbacks = [tensorboard])

Epoch 1/80
14/14 [==============================] - 2s 38ms/step - loss: 1.6828 - auc_13: 0.7891 - precision_13: 0.1732 - recall_13: 0.7985 - accuracy: 0.0000e+00 - val_loss: 1.3200 - val_auc_13: 0.8667 - val_precision_13: 0.7329 - val_recall_13: 0.2074 - val_accuracy: 0.0000e+00
Epoch 2/80
14/14 [==============================] - 0s 14ms/step - loss: 1.4327 - auc_13: 0.8309 - precision_13: 0.2214 - recall_13: 0.7890 - accuracy: 0.0000e+00 - val_loss: 1.1736 - val_auc_13: 0.8967 - val_precision_13: 1.0000 - val_recall_13: 0.0018 - val_accuracy: 0.0000e+00
Epoch 3/80
14/14 [==============================] - 0s 14ms/step - loss: 1.2921 - auc_13: 0.8573 - precision_13: 0.2602 - recall_13: 0.7892 - accuracy: 0.0000e+00 - val_loss: 1.0554 - val_auc_13: 0.9046 - val_precision_13: 0.0000e+00 - val_recall_13: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/80
14/14 [==============================] - 0s 14ms/step - loss: 1.1743 - auc_13: 0.8742 - precision_13: 0.2984 - recall_13: 0.7870 - accurac

In [76]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve
n_splits = 5  # Number of folds for StratifiedKFold
beta= 1.45
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

opt_thresholds = []
f1_scores = []

for train_index, val_index in cv.split(combined_X_train, combined_y_train):
    # Split data into training and validation for the current fold
    X_train_curr, X_val_curr = combined_X_train[train_index], combined_X_train[val_index]
    y_train_curr, y_val_curr = combined_y_train[train_index], combined_y_train[val_index]

    # Predict probabilities for the positive class on the validation data
    y_pred_prob = model.predict(X_val_curr).ravel()

    # Compute precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_val_curr, y_pred_prob)

    # Calculate F1 scores for each threshold
    # f1_scores_fold = [2 * (p * r) / (p + r) if (p + r) > 0 else 0 for p, r in zip(precision, recall)]
    f1_scores_fold = [(1 + beta**2) * (prec * rec) / ((beta**2 * prec) + rec) for prec, rec in zip(precision, recall)]

    # Find the index of the maximum F1 score
    opt_idx = np.argmax(f1_scores_fold)
    opt_threshold = thresholds[opt_idx] if opt_idx < len(thresholds) else 1.0

    # Store the optimal threshold for this fold
    opt_thresholds.append(opt_threshold)
    f1_scores.append(f1_scores_fold[opt_idx])

# Optionally, you can print or analyze the optimal thresholds and F1 scores for each fold
for i, (threshold, f1_score) in enumerate(zip(opt_thresholds, f1_scores)):
    print(f"Fold {i+1}: Optimal threshold: {threshold}, F1 Score: {f1_score}")

# You might also calculate and print the average optimal threshold and F1 score across folds if desired
average_opt_threshold = np.mean(opt_thresholds)
average_f1_score = np.mean(f1_scores)
print(f"Average Optimal Threshold: {average_opt_threshold}, Average F1 Score: {average_f1_score}")

416/416 [==============================] - 1s 2ms/step
Fold 1: Optimal threshold: 0.5391228199005127, F1 Score: 0.7596661028649636
Fold 2: Optimal threshold: 0.5223883390426636, F1 Score: 0.7383446393231858
Fold 3: Optimal threshold: 0.5060943961143494, F1 Score: 0.7415705021453342
Fold 4: Optimal threshold: 0.5399326086044312, F1 Score: 0.7735631880635606
Fold 5: Optimal threshold: 0.5187482833862305, F1 Score: 0.7567127746135069
Average Optimal Threshold: 0.5252572894096375, Average F1 Score: 0.7539714414021101


In [77]:
y_pred_prob = (model.predict(X_val_scaled) > average_opt_threshold).astype(int)
print(classification_report(y_val, y_pred_prob))
y_pred_prob = (model.predict(X_test_scaled).ravel() >= average_opt_threshold).astype(int)
print(classification_report(y_test, y_pred_prob))

367/367 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     10658
           1       0.77      0.75      0.76      1085

    accuracy                           0.96     11743
   macro avg       0.87      0.86      0.87     11743
weighted avg       0.96      0.96      0.96     11743
367/367 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     10658
           1       0.74      0.73      0.74      1084

    accuracy                           0.95     11742
   macro avg       0.86      0.85      0.85     11742
weighted avg       0.95      0.95      0.95     11742


In [16]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [17]:
# Create a converter object
ae = tf.keras.models.load_model('model')
converter = tf.lite.TFLiteConverter.from_keras_model(ae)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]
# This example enables dynamic range quantization
converter.allow_custom_ops = True  # Allow for the possibility of custom operations

# Enable verbose logging
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]


# Perform the conversion
tflite_model = converter.convert()
# Replace 'converted_model.tflite' with the desired path for your .tflite model
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpf4066eal/assets


INFO:tensorflow:Assets written to: /tmp/tmpf4066eal/assets
2024-02-23 02:15:57.231097: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-23 02:15:57.231125: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-23 02:15:57.231388: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpf4066eal
2024-02-23 02:15:57.232453: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-23 02:15:57.232461: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpf4066eal
2024-02-23 02:15:57.235180: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-23 02:15:57.236013: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-23 02:15:57.258547: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio